# Leitlinien RAG-System

# Mistral API Initialisierung

Zunächst wird die API von Mistral initialisiert, da das Sprachmodell für den Retrieval-Teil benötigt wird.

In [2]:
import dotenv
import os
from langchain_core.messages import HumanMessage
from langchain_mistralai.chat_models import ChatMistralAI

dotenv.load_dotenv()


api_key = os.getenv("MISTRAL_API_KEY")
llm = ChatMistralAI(api_key=api_key)

## Metadaten

Die Metadaten werden aus der beim Scrapen erstellten PDF ausgelesen und so angepasst, dass sie für die weitere Verarbeitung bereit sind.

In [9]:
with open("metadata.json", "r") as f:
    metadata = pd.read_json(f).T
    metadata['Registiernummer'] = metadata['Registiernummer'].str.replace("Registernummer ", "")
    metadata['Federführende Fachgesellschaft(en)'] = metadata['Federführende Fachgesellschaft(en)'].str.split(", ")
    metadata['Adressaten'] = metadata['Adressaten'].str.split(", ")
    metadata['Patientenzielgruppe'] = metadata['Patientenzielgruppe'].str.split(", ")
    metadata['Versorgungsbereich'] = metadata['Versorgungsbereich'].str.split(", ")
    metadata['Schlüsselwörter'] = metadata['Schlüsselwörter'].str.split(", ")
    metadata['pdf_links'] = 'https://register.awmf.org' + metadata['pdf_links'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
    metadata.drop(columns=['Patienteninformation', 'Zielorientierung der Leitlinie', 'Gründe für die Themenwahl',  ], inplace=True)
    metadata = metadata.to_dict(orient="index")

# Vorbereitung der PDFs

Bevor die PDFs indexiert werden können, müssen sie zunächst vorbereitet werden. Hierfür wird die Bibliothek pdfplumber verwendet. Damit kann der Text extrahiert und die Seitenzahl in den Metadaten gespeichert werden. 

In [12]:

import pdfplumber
import pandas as pd
from tqdm import tqdm
from langchain.schema import Document  # or whatever you're using
import pdfplumber
import pandas as pd
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.utils import filter_complex_metadata
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma

model = SentenceTransformer("intfloat/multilingual-e5-large-instruct", device="mps")  # Apple Silicon acceleration

def normalize(text):
    """Remove extra whitespace and normalize newlines for better matching."""
    return ' '.join(text.replace("\n", " ").split())

def load_pdf_with_tables(path, metadata):
    docs = []
    previous_table = None
    previous_header = None

    identifier = path.split("/")[-1].split(".")[0]
    pdf_metadata = metadata.get(identifier, {})

    with pdfplumber.open(path) as pdf:
        for i, page in enumerate(pdf.pages):
            raw_text = page.extract_text() or ""
            text = normalize(raw_text)

            full_text = text.strip()
            
            page_metadata = pdf_metadata.copy()
            page_metadata.update({"page": i + 1})
            docs.append(Document(page_content=full_text.strip(), metadata=page_metadata))

    return docs

def chunk_list(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]


## Indexierung aller Leitlinien

Die Leitlinien sind in Abschnitte von jeweils etwa 1.000 Token unterteilt. Die Registriernummern der indexierten Dateien werden in einer Textdatei gespeichert, um eine mehrfache Indexierung zu verhindern, falls die Indexierung unerwartet unterbrochen wird. Dateien, bei denen die Indexierung gescheitert ist, werden ebenfalls in einer Textdatei abgespeichert, um es später erneut versuchen zu können.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
# ----------------------------
# Document splitter setup
# ----------------------------
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""],
)

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct",
    show_progress=True,
)

vector_store = Chroma(
    collection_name="guidelines",
    embedding_function=embeddings,
    persist_directory="/Volumes/FESTPLATTE/sma_project/chroma_langchain_db",
)

# ----------------------------
# Indexing loop
# ----------------------------
base_dir = "/Volumes/FESTPLATTE/sma_project/guidelines/"
max_batch_size = 5000

with open("indexed_guidelines.txt", "r") as f:
    indexed_ids = set(f.read().splitlines())  # use set for faster lookup

for idd in metadata.keys():
    if idd in indexed_ids:
        continue

    try:
        pdf_path = f"{base_dir}/{idd}.pdf"
        docs = load_pdf_with_tables(pdf_path, metadata)

        docs_split = splitter.split_documents(docs)
        
        # add complex metadata filtering
        filtered_docs = [
            Document(
                page_content=doc.page_content,
                metadata=filter_complex_metadata([doc])[0].metadata
            )
            for doc in docs_split
        ]
        
        texts = [doc.page_content for doc in filtered_docs]
        metadatas = [doc.metadata for doc in filtered_docs]
        
        # creation of the embedding
        embeddings_list = model.encode(texts, show_progress_bar=False, batch_size=8, convert_to_tensor=True).tolist()
        
        # chunking the embeddings to avoid memory issues
        for b_i, batch in enumerate(chunk_list(embeddings_list, max_batch_size)):
            vector_store._collection.add(
                embeddings=batch,
                documents=texts[:len(batch)],
                metadatas=metadatas[:len(batch)],
                ids=[f"{idd}-{b_i}-{i}" for i in range(len(batch))]
            )

        with open("indexed_guidelines.txt", "a") as f:
            f.write(f"{idd}\n")

    except Exception as e:
        print(f"Failed to process {idd}: {e}")
        with open("failed_guidelines.txt", "a") as f:
            f.write(f"{idd}\n")
